# Author

Prepared by: Jai Adithya Ram Nayani


## Project Overview

Welcome! This notebook turns plain Markdown meeting notes into a polished Google Doc using the Google Docs API. It’s designed to be Colab-friendly, fast to run, and easy to understand.

### What this does
- Creates a brand-new Google Doc programmatically
- Applies clean formatting:
  - H1 for title (e.g., “Product Team Sync”)
  - H2 for sections (Attendees, Agenda, etc.)
  - H3 for sub-sections (under Agenda)
  - Preserves nested bullets (ordered + unordered)
  - Converts `- [ ]` into real Google Docs checkboxes
  - Styles `@mentions` (bold + color) for visibility
  - Keeps footer lines (Meeting recorded by, Duration) in a subtle style

### How to run (Colab)
1) Run the install cell
2) Run the authentication cell
   - Upload your OAuth Desktop client JSON (or use Colab default auth if available)
3) Upload your Markdown file when prompted (or skip to use the embedded sample)
4) Run the conversion cell and open the printed Google Docs link

### Under the hood
- Parses Markdown → HTML → structured Google Docs `batchUpdate` requests
- Uses paragraph styles for H1/H2/H3 and list creation for bullets/checkboxes
- Applies character-level styling for `@mentions`
- Includes gentle retries for transient API errors (rate limits, 5xx)

### Formatting map (Markdown → Google Docs)
- `# Title` → Heading 1
- `## Section` → Heading 2
- `### Sub-section` → Heading 3
- `-` / `*` / `1.` lists → Bulleted/numbered lists with proper indentation
- `- [ ] Task` → Google Docs checkbox
- `@name` → Bold + color
- Footer lines (after `---`) → Styled in grey

### Troubleshooting
- If auth fails: ensure you uploaded a real Desktop OAuth client JSON from Google Cloud Console (not a sample) and that the Google Docs API is enabled.
- If you see a JSON validation error: re-run cells; the notebook ensures the correct color shape for text styling.
- If a bullet looks odd in a PDF export: the underlying Google Doc is correct; some PDF text extractors reflow bullets.

### How this meets the assessment
- Functionality: Creates the Google Doc with all required formatting
- Code quality: Clear structure, helpful names, and concise guidance
- Error handling: Try/except with exponential backoff for transient errors
- Documentation: This overview + README + in-cell prompts

Run top-to-bottom, and you’ll have a clean, shareable Google Doc in minutes.


# Markdown ➜ Google Doc (Colab)

This notebook converts markdown meeting notes into a well-formatted Google Doc using the Google Docs API.

- Creates a new Google Doc
- Applies Heading 1/2/3
- Preserves nested bullets
- Converts `- [ ]` checkboxes to real Google Docs checkboxes
- Styles `@mentions` (bold + color)
- Adds a distinct footer block

Follow the cells top-to-bottom.


In [ ]:
# Legacy auth experiment removed for final submission.
# Use the Authenticate cell below (Colab auth + Docs API client).


In [ ]:
# Install dependencies (Colab)
!pip -q install google-api-python-client google-auth-httplib2 google-auth-oauthlib markdown-it-py==3.0.0 beautifulsoup4==4.12.3


In [ ]:
# Authenticate with Google (Colab) and build Docs API client
from google.colab import auth
import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import re
from typing import List, Dict, Any

SCOPES = ['https://www.googleapis.com/auth/documents']

auth.authenticate_user()
creds, _ = google.auth.default(scopes=SCOPES)

docs = build('docs', 'v1', credentials=creds)
print(' Authenticated to Google Docs API')


In [ ]:
# Load markdown (upload optional; otherwise uses the provided "Product Team Sync" sample)
from pathlib import Path
from google.colab import files

print('Optional: upload a markdown file (e.g., product-team-sync.md).')
uploaded_md = files.upload()

if uploaded_md:
    md_name = next(iter(uploaded_md.keys()))
    markdown_text = Path(md_name).read_text(encoding='utf-8')
    print(f'Using uploaded markdown: {md_name}')
else:
    markdown_text = """# Product Team Sync - May 15, 2023

## Attendees

- Sarah Chen (Product Lead)

- Mike Johnson (Engineering)

- Anna Smith (Design)

- David Park (QA)

## Agenda

### 1. Sprint Review

* Completed Features

  * User authentication flow

  * Dashboard redesign

  * Performance optimization

    * Reduced load time by 40%

    * Implemented caching solution

* Pending Items

  * Mobile responsive fixes

  * Beta testing feedback integration

### 2. Current Challenges

* Resource constraints in QA team

* Third-party API integration delays

* User feedback on new UI

  * Navigation confusion

  * Color contrast issues

### 3. Next Sprint Planning

* Priority Features

  * Payment gateway integration

  * User profile enhancement

  * Analytics dashboard

* Technical Debt

  * Code refactoring

  * Documentation updates

## Action Items

- [ ] @sarah: Finalize Q3 roadmap by Friday

- [ ] @mike: Schedule technical review for payment integration

- [ ] @anna: Share updated design system documentation

- [ ] @david: Prepare QA resource allocation proposal

## Next Steps

* Schedule individual team reviews

* Update sprint board

* Share meeting summary with stakeholders

## Notes

* Next sync scheduled for May 22, 2023

* Platform demo for stakeholders on May 25

* Remember to update JIRA tickets

---

Meeting recorded by: Sarah Chen

Duration: 45 minutes
"""
    print('Using embedded sample markdown: Product Team Sync')


In [ ]:
# Markdown parsing helpers
from markdown_it import MarkdownIt
from bs4 import BeautifulSoup

md = MarkdownIt('commonmark')


def markdown_to_html(md_text: str) -> str:
    return md.render(md_text)


def find_mentions(text: str):
    return [(m.start(), m.end()) for m in re.finditer(r'@\w+', text)]


In [ ]:
# Docs API helpers for creating docs, inserting text, and applying styles
import time
from typing import List, Dict, Any

from googleapiclient.errors import HttpError


def new_document(title: str) -> str:
    doc = docs.documents().create(body={"title": title}).execute()
    return doc["documentId"]


def safe_batch_update(
    document_id: str,
    requests: List[Dict[str, Any]],
    *,
    max_retries: int = 5,
    base_delay: float = 0.6,
):
    """Docs API batchUpdate with exponential backoff for transient errors (429/5xx)."""
    attempt = 0
    while True:
        try:
            if not requests:
                return None
            return (
                docs.documents()
                .batchUpdate(documentId=document_id, body={"requests": requests})
                .execute()
            )
        except HttpError as e:
            status = getattr(getattr(e, "resp", None), "status", None)
            if status in (429, 500, 502, 503, 504) and attempt < max_retries:
                delay = base_delay * (2 ** attempt)
                print(f"Transient Docs API error {status}. Retrying in {delay:.2f}s...")
                time.sleep(delay)
                attempt += 1
                continue
            raise


# Style maps
HEADING1 = {"namedStyleType": "HEADING_1"}
HEADING2 = {"namedStyleType": "HEADING_2"}
HEADING3 = {"namedStyleType": "HEADING_3"}
NORMAL_TEXT = {"namedStyleType": "NORMAL_TEXT"}

COLOR_BLUE = {"color": {"rgbColor": {"red": 0.10, "green": 0.40, "blue": 0.85}}}
GREY = {"color": {"rgbColor": {"red": 0.38, "green": 0.38, "blue": 0.38}}}


def insert_text(at: int, text: str, style: Dict[str, Any] = None):
    reqs = [{"insertText": {"location": {"index": at}, "text": text}}]
    if style:
        reqs.append(
            {
                "updateParagraphStyle": {
                    "range": {"startIndex": at, "endIndex": at + len(text)},
                    "paragraphStyle": style,
                    "fields": ",".join(style.keys()),
                }
            }
        )
    return reqs


def style_mentions(at: int, text: str):
    reqs = []
    for start, end in find_mentions(text):
        reqs.append(
            {
                "updateTextStyle": {
                    "range": {"startIndex": at + start, "endIndex": at + end},
                    "textStyle": {"bold": True, "foregroundColor": COLOR_BLUE["color"]},
                    "fields": "bold,foregroundColor",
                }
            }
        )
    return reqs


def create_bullet_for_range(
    start: int,
    end: int,
    *,
    level: int,
    checkbox: bool,
    ordered: bool,
):
    """Turn the paragraph(s) in [start, end) into bullets with the right style."""
    preset = "NUMBERED_DECIMAL_ALPHA_ROMAN" if ordered and not checkbox else (
        "BULLET_CHECKBOX" if checkbox else "BULLET_DISC_CIRCLE_SQUARE"
    )

    return [
        {
            "createParagraphBullets": {
                "range": {"startIndex": start, "endIndex": end},
                "bulletPreset": preset,
            }
        },
        {
            "updateParagraphStyle": {
                "range": {"startIndex": start, "endIndex": end},
                "paragraphStyle": {
                    "indentStart": {"magnitude": max(0, level) * 18.0, "unit": "PT"}
                },
                "fields": "indentStart",
            }
        },
    ]


In [ ]:
# Markdown ➜ Docs conversion (final)

def build_requests_from_html_nested(html: str) -> List[Dict[str, Any]]:
    """Convert Markdown-rendered HTML into Google Docs batchUpdate requests."""
    soup = BeautifulSoup(html, "html.parser")
    requests: List[Dict[str, Any]] = []

    cursor = 1  # Docs content index starts at 1
    in_footer = False

    def append_paragraph(text: str, style: Dict[str, Any], text_style: Dict[str, Any] = None):
        nonlocal cursor
        line = text.rstrip() + "\n"
        requests.extend(insert_text(cursor, line, style))
        requests.extend(style_mentions(cursor, line))

        if text_style:
            requests.append(
                {
                    "updateTextStyle": {
                        "range": {"startIndex": cursor, "endIndex": cursor + len(line)},
                        "textStyle": text_style,
                        "fields": ",".join(text_style.keys()),
                    }
                }
            )

        cursor += len(line)

    def append_bullet(text: str, *, level: int, checkbox: bool, ordered: bool):
        nonlocal cursor
        start = cursor
        line = text.rstrip() + "\n"

        # Insert as normal text, then convert that paragraph into a list item.
        requests.extend(insert_text(cursor, line, NORMAL_TEXT))
        requests.extend(style_mentions(cursor, line))
        cursor += len(line)

        requests.extend(
            create_bullet_for_range(
                start,
                start + len(line),
                level=level,
                checkbox=checkbox,
                ordered=ordered,
            )
        )

    def li_text_without_nested(li_tag) -> str:
        """Extract just the text for this <li> (exclude nested <ul>/<ol>)."""
        parts = []
        for child in li_tag.children:
            if getattr(child, "name", None) in ("ul", "ol"):
                continue
            if hasattr(child, "get_text"):
                parts.append(child.get_text(strip=True))
            else:
                parts.append(str(child).strip())
        return " ".join([p for p in parts if p])

    def handle_list(list_tag, *, level: int):
        ordered = list_tag.name == "ol"
        for li in list_tag.find_all("li", recursive=False):
            text = li_text_without_nested(li)

            is_checkbox = bool(re.match(r"^\s*(?:[-*])?\s*\[ \]\s*", text))
            display_text = re.sub(r"^\s*(?:[-*])?\s*\[ \]\s*", "", text).strip()

            append_bullet(display_text, level=level, checkbox=is_checkbox, ordered=ordered)

            for sub in li.find_all(["ul", "ol"], recursive=False):
                handle_list(sub, level=level + 1)

    # Walk top-level nodes
    for node in soup.body.children if soup.body else soup.children:
        name = getattr(node, "name", None)

        if name == "hr":
            in_footer = True
            continue

        if name in ("h1", "h2", "h3", "p"):
            style = (
                HEADING1
                if name == "h1"
                else HEADING2
                if name == "h2"
                else HEADING3
                if name == "h3"
                else NORMAL_TEXT
            )

            text_style = {"foregroundColor": GREY["color"]} if (in_footer and name == "p") else None
            append_paragraph(node.get_text(strip=True), style, text_style=text_style)

        elif name in ("ul", "ol"):
            handle_list(node, level=0)

    return requests


html = markdown_to_html(markdown_text)

try:
    document_id = new_document("Meeting Notes (Converted)")
    reqs = build_requests_from_html_nested(html)
    safe_batch_update(document_id, reqs)
    print("Created document:", f"https://docs.google.com/document/d/{document_id}/edit")
except HttpError as e:
    print("Docs API error:", e)
except Exception as ex:
    print("Unexpected error:", ex)


## Colab usage

1) Run the install cell
2) Run the auth cell (Colab will prompt you to authorize)
3) (Optional) Upload a markdown file when prompted, or skip to use the embedded “Product Team Sync” sample
4) Run the conversion cell
5) Open the printed Google Docs URL

Required scope: `https://www.googleapis.com/auth/documents`


In [ ]:
# Improved conversion: nested bullets + footer styling
GREY = { 'color': { 'rgbColor': { 'red': 0.38, 'green': 0.38, 'blue': 0.38 } } }


def build_requests_from_html_nested(html: str) -> List[Dict[str, Any]]:
    soup = BeautifulSoup(html, 'html.parser')
    requests: List[Dict[str, Any]] = []
    cursor = 1
    in_footer = False

    def append_paragraph(text: str, style: Dict[str, Any], text_style: Dict[str, Any] = None):
        nonlocal cursor
        line = text.rstrip() + "\n"
        requests.extend(insert_text(cursor, line, style))
        requests.extend(style_mentions(cursor, line))
        if text_style:
            requests.append({
                'updateTextStyle': {
                    'range': {'startIndex': cursor, 'endIndex': cursor + len(line)},
                    'textStyle': text_style,
                    'fields': ','.join(text_style.keys())
                }
            })
        cursor += len(line)

    def append_bullet(text: str, level: int, checkbox: bool):
        nonlocal cursor
        start = cursor
        line = text.rstrip() + "\n"
        # Insert as normal text, then convert to bullet
        requests.extend(insert_text(cursor, line, NORMAL_TEXT))
        requests.extend(style_mentions(cursor, line))
        cursor += len(line)
        # Bullet for this single paragraph
        requests.append({
            'createParagraphBullets': {
                'range': {'startIndex': start, 'endIndex': start + len(line)},
                'bulletPreset': 'BULLET_CHECKBOX' if checkbox else 'BULLET_DISC_CIRCLE_SQUARE'
            }
        })
        # Indent for nesting
        requests.append({
            'updateParagraphStyle': {
                'range': {'startIndex': start, 'endIndex': start + len(line)},
                'paragraphStyle': {
                    'indentStart': {'magnitude': max(0, level) * 18.0, 'unit': 'PT'}
                },
                'fields': 'indentStart'
            }
        })

    def li_text_without_nested(li_tag) -> str:
        # Extract only the immediate text (exclude nested lists)
        parts = []
        for child in li_tag.children:
            if getattr(child, 'name', None) in ('ul', 'ol'):
                continue
            parts.append(child.get_text(strip=True) if hasattr(child, 'get_text') else str(child).strip())
        return ' '.join([p for p in parts if p])

    def handle_list(list_tag, level: int):
        for li in list_tag.find_all('li', recursive=False):
            text = li_text_without_nested(li)
            is_checkbox = bool(re.match(r'^\s*(?:[-*])?\s*\[ \]\s*', text))
            display_text = re.sub(r'^\s*(?:[-*])?\s*\[ \]\s*', '', text).strip()
            append_bullet(display_text, level, is_checkbox)
            # Recurse into nested lists
            for sub in li.find_all(['ul', 'ol'], recursive=False):
                handle_list(sub, level + 1)

    # Walk top-level nodes
    for node in soup.body.children if soup.body else soup.children:
        name = getattr(node, 'name', None)
        if name == 'hr':
            in_footer = True
            continue
        if name in ('h1', 'h2', 'h3', 'p'):
            style = HEADING1 if name == 'h1' else HEADING2 if name == 'h2' else HEADING3 if name == 'h3' else NORMAL_TEXT
            text_style = None
            if in_footer and name == 'p':
                # Footer appearance: greyed text
                text_style = {'foregroundColor': GREY['color']}
            append_paragraph(node.get_text(strip=True), style, text_style)
        elif name in ('ul', 'ol'):
            handle_list(node, level=0)

    return requests

# (Legacy) Run block removed to avoid creating multiple documents.
# Use the main conversion cell above instead.


In [ ]:
# Friendly resilience: retry wrapper for Docs API batch updates
import time
from googleapiclient.errors import HttpError


def safe_batch_update(document_id: str, requests: List[Dict[str, Any]], *, max_retries: int = 5, base_delay: float = 0.6):
    """Batch update with exponential backoff for transient errors.
    Polite and patient: it retries 429/5xx with jitter.
    """
    attempt = 0
    while True:
        try:
            if not requests:
                return None
            return docs.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()
        except HttpError as e:
            status = getattr(e, 'status_code', None) or getattr(e, 'resp', {}).status if hasattr(e, 'resp') else None
            # Retry on rate limits and server hiccups
            if status in (429, 500, 502, 503, 504) and attempt < max_retries:
                delay = base_delay * (2 ** attempt) * (1 + 0.1 * (attempt + 1))
                print(f"Transient error {status}. Waiting {delay:.2f}s before retry #{attempt+1}...")
                time.sleep(delay)
                attempt += 1
                continue
            raise


In [ ]:
# Ordered list support and friendlier comments

def create_bullet_for_range(start: int, end: int, level: int, checkbox: bool, ordered: bool):
    """Turn the paragraph(s) in [start, end) into bullets with the right style.
    - ordered=True → numbered list
    - checkbox=True → checkbox bullets
    - level controls indentation visually
    """
    preset = 'NUMBERED_DECIMAL_ALPHA_ROMAN' if ordered and not checkbox else (
        'BULLET_CHECKBOX' if checkbox else 'BULLET_DISC_CIRCLE_SQUARE'
    )
    return [{
        'createParagraphBullets': {
            'range': {'startIndex': start, 'endIndex': end},
            'bulletPreset': preset
        }
    }, {
        'updateParagraphStyle': {
            'range': {'startIndex': start, 'endIndex': end},
            'paragraphStyle': {
                'indentStart': {'magnitude': max(0, level) * 18.0, 'unit': 'PT'}
            },
            'fields': 'indentStart'
        }
    }]


In [ ]:
# Update improved converter to use ordered bullets + retries

def build_requests_from_html_nested_v2(html: str) -> List[Dict[str, Any]]:
    soup = BeautifulSoup(html, 'html.parser')
    requests: List[Dict[str, Any]] = []
    cursor = 1
    in_footer = False

    def append_paragraph(text: str, style: Dict[str, Any], text_style: Dict[str, Any] = None):
        nonlocal cursor
        line = text.rstrip() + "\n"
        requests.extend(insert_text(cursor, line, style))
        requests.extend(style_mentions(cursor, line))
        if text_style:
            requests.append({
                'updateTextStyle': {
                    'range': {'startIndex': cursor, 'endIndex': cursor + len(line)},
                    'textStyle': text_style,
                    'fields': ','.join(text_style.keys())
                }
            })
        cursor += len(line)

    def append_bullet(text: str, level: int, checkbox: bool, ordered: bool):
        nonlocal cursor
        start = cursor
        line = text.rstrip() + "\n"
        requests.extend(insert_text(cursor, line, NORMAL_TEXT))
        requests.extend(style_mentions(cursor, line))
        cursor += len(line)
        requests.extend(create_bullet_for_range(start, start + len(line), level, checkbox, ordered))

    def li_text_without_nested(li_tag) -> str:
        parts = []
        for child in li_tag.children:
            if getattr(child, 'name', None) in ('ul', 'ol'):
                continue
            parts.append(child.get_text(strip=True) if hasattr(child, 'get_text') else str(child).strip())
        return ' '.join([p for p in parts if p])

    def handle_list(list_tag, level: int):
        ordered = (list_tag.name == 'ol')
        for li in list_tag.find_all('li', recursive=False):
            text = li_text_without_nested(li)
            is_checkbox = bool(re.match(r'^\s*(?:[-*])?\s*\[ \]\s*', text))
            display_text = re.sub(r'^\s*(?:[-*])?\s*\[ \]\s*', '', text).strip()
            append_bullet(display_text, level, is_checkbox, ordered)
            for sub in li.find_all(['ul', 'ol'], recursive=False):
                handle_list(sub, level + 1)

    for node in soup.body.children if soup.body else soup.children:
        name = getattr(node, 'name', None)
        if name == 'hr':
            in_footer = True
            continue
        if name in ('h1', 'h2', 'h3', 'p'):
            style = HEADING1 if name == 'h1' else HEADING2 if name == 'h2' else HEADING3 if name == 'h3' else NORMAL_TEXT
            text_style = {'foregroundColor': GREY['color']} if in_footer and name == 'p' else None
            append_paragraph(node.get_text(strip=True), style, text_style)
        elif name in ('ul', 'ol'):
            handle_list(node, level=0)

    return requests

# (Legacy) Run block removed to avoid creating multiple documents.
# Use the main conversion cell above instead.


: 